# Classificação de Variedades de Grãos de Trigo
## Aplicação da Metodologia CRISP-DM

**Integrantes do Grupo:**
- Vinicius de Santana Gama - RM566672
- Pedro Carvalho Rocha Lima - RM567330
- Vinicius Lisboa Porto - RM561406
- Marlon Paulino Marinho - RM566793
- Danilo Marques Dantas - RM567583

---

## Contexto do Problema

Em cooperativas agrícolas de pequeno porte, a classificação manual de grãos é demorada e sujeita a erros humanos. Este projeto desenvolve um modelo de aprendizado de máquina para automatizar a classificação de três variedades de trigo (Kama, Rosa e Canadian) com base em características físicas dos grãos.

**Objetivo:** Aplicar CRISP-DM para desenvolver, comparar e otimizar modelos de classificação que auxiliem cooperativas agrícolas na automação do processo de classificação de grãos.

## 📁 IMPORTANTE: Upload do Dataset

**ANTES DE CONTINUAR:**
1. Clique no ícone de **pasta** 📁 no menu lateral esquerdo
2. Clique no ícone de **upload** (seta para cima)
3. Selecione o arquivo `seeds_dataset.txt`
4. Aguarde o upload completar
5. Execute as células abaixo

## 1. Importação de Bibliotecas

In [ ]:
# Manipulação de dados
import pandas as pd
import numpy as np
import os

# Visualização
import matplotlib.pyplot as plt
import seaborn as sns

# Pré-processamento
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.preprocessing import StandardScaler

# Algoritmos de classificação
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression

# Métricas de avaliação
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    confusion_matrix, classification_report, ConfusionMatrixDisplay
)

# Configurações de visualização
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette('husl')
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 10

# Ignorar warnings desnecessários
import warnings
warnings.filterwarnings('ignore')

print("✓ Bibliotecas importadas com sucesso")

## 2. Carregamento e Exploração Inicial dos Dados

In [ ]:
# Definição dos nomes das colunas conforme documentação
column_names = [
    'Area',
    'Perimetro',
    'Compacidade',
    'Comprimento_Nucleo',
    'Largura_Nucleo',
    'Coef_Assimetria',
    'Comprimento_Sulco',
    'Variedade'
]

# Buscar arquivo em possíveis localizações
possible_paths = [
    'seeds_dataset.txt',  # Diretório atual
    '/content/seeds_dataset.txt',  # Raiz do Colab
    'sample_data/seeds_dataset.txt'  # Pasta sample_data
]

file_path = None
for path in possible_paths:
    if os.path.exists(path):
        file_path = path
        print(f"✓ Arquivo encontrado em: {path}")
        break

if file_path is None:
    print("❌ ERRO: Arquivo seeds_dataset.txt não encontrado!")
    print("\n📋 INSTRUÇÕES:")
    print("1. Clique no ícone de PASTA 📁 no menu lateral esquerdo")
    print("2. Clique no ícone de UPLOAD (seta para cima)")
    print("3. Selecione o arquivo 'seeds_dataset.txt'")
    print("4. Aguarde o upload completar")
    print("5. Execute esta célula novamente")
    raise FileNotFoundError("seeds_dataset.txt não encontrado")

# Carregamento do dataset
# Usando delimitador de espaços em branco (tabs ou espaços múltiplos)
df = pd.read_csv(
    file_path,
    sep='\s+',  # Aceita um ou mais espaços/tabs
    names=column_names,
    header=None,
    engine='python'  # Necessário para usar regex no sep
)

print(f"\n✓ Dataset carregado com sucesso!")
print(f"Dimensões: {df.shape[0]} amostras x {df.shape[1]} colunas")
print(f"\n{'='*60}")
print("PRIMEIRAS LINHAS DO DATASET")
print(f"{'='*60}")
df.head(10)

In [ ]:
# Informações gerais do dataset
print("INFORMAÇÕES GERAIS DO DATASET")
print(f"{'='*60}")
df.info()

print(f"\n{'='*60}")
print("DISTRIBUIÇÃO DAS CLASSES")
print(f"{'='*60}")
print(df['Variedade'].value_counts().sort_index())
print(f"\nClasses balanceadas: {df['Variedade'].value_counts().std() < 5}")

In [ ]:
# Mapeamento das variedades
variedades_map = {1: 'Kama', 2: 'Rosa', 3: 'Canadian'}
print("\nMapeamento das Variedades:")
for key, value in variedades_map.items():
    count = (df['Variedade'] == key).sum()
    print(f"  {key} = {value} ({count} amostras)")

## 3. Análise Exploratória de Dados (EDA)

### 3.1 Estatísticas Descritivas

In [ ]:
# Estatísticas descritivas completas
print("ESTATÍSTICAS DESCRITIVAS")
print(f"{'='*60}")
df.describe().T

In [ ]:
# Verificação de valores ausentes
print("VERIFICAÇÃO DE VALORES AUSENTES")
print(f"{'='*60}")
missing = df.isnull().sum()
print(missing)
print(f"\n✓ Total de valores ausentes: {missing.sum()}")

### 3.2 Visualização da Distribuição das Features

In [ ]:
# Histogramas das features
features = df.columns[:-1]

fig, axes = plt.subplots(3, 3, figsize=(16, 12))
axes = axes.ravel()

for idx, col in enumerate(features):
    axes[idx].hist(df[col], bins=25, edgecolor='black', alpha=0.7)
    axes[idx].set_title(f'Distribuição: {col}', fontweight='bold')
    axes[idx].set_xlabel(col)
    axes[idx].set_ylabel('Frequência')
    axes[idx].grid(True, alpha=0.3)

# Remover subplots vazios
fig.delaxes(axes[7])
fig.delaxes(axes[8])

plt.tight_layout()
plt.suptitle('Distribuição das Características dos Grãos', fontsize=16, fontweight='bold', y=1.01)
plt.show()

### 3.3 Análise de Outliers com Boxplots

In [ ]:
# Boxplots para identificar outliers
fig, axes = plt.subplots(3, 3, figsize=(16, 12))
axes = axes.ravel()

for idx, col in enumerate(features):
    axes[idx].boxplot(df[col], vert=True, patch_artist=True,
                     boxprops=dict(facecolor='lightblue', alpha=0.7),
                     medianprops=dict(color='red', linewidth=2))
    axes[idx].set_title(f'Boxplot: {col}', fontweight='bold')
    axes[idx].set_ylabel(col)
    axes[idx].grid(True, alpha=0.3)

fig.delaxes(axes[7])
fig.delaxes(axes[8])

plt.tight_layout()
plt.suptitle('Análise de Outliers - Boxplots', fontsize=16, fontweight='bold', y=1.01)
plt.show()

### 3.4 Matriz de Correlação

In [ ]:
# Matriz de correlação
correlation_matrix = df[features].corr()

plt.figure(figsize=(10, 8))
sns.heatmap(correlation_matrix, annot=True, fmt='.2f', cmap='coolwarm',
            square=True, linewidths=0.5, cbar_kws={'shrink': 0.8})
plt.title('Matriz de Correlação entre Features', fontsize=14, fontweight='bold', pad=20)
plt.tight_layout()
plt.show()

# Identificar correlações fortes
print("\nCORRELAÇÕES FORTES (|r| > 0.8):")
print(f"{'='*60}")
for i in range(len(correlation_matrix.columns)):
    for j in range(i+1, len(correlation_matrix.columns)):
        if abs(correlation_matrix.iloc[i, j]) > 0.8:
            print(f"{correlation_matrix.columns[i]:25} <-> {correlation_matrix.columns[j]:25} = {correlation_matrix.iloc[i, j]:.3f}")

### 3.5 Scatter Plots - Relações entre Features

In [ ]:
# Scatter plots das principais relações
principais_relacoes = [
    ('Area', 'Perimetro'),
    ('Comprimento_Nucleo', 'Largura_Nucleo'),
    ('Area', 'Compacidade'),
    ('Perimetro', 'Comprimento_Sulco')
]

fig, axes = plt.subplots(2, 2, figsize=(14, 10))
axes = axes.ravel()

cores = {1: 'red', 2: 'blue', 3: 'green'}
labels = {1: 'Kama', 2: 'Rosa', 3: 'Canadian'}

for idx, (feat1, feat2) in enumerate(principais_relacoes):
    for variedade in [1, 2, 3]:
        mask = df['Variedade'] == variedade
        axes[idx].scatter(df[mask][feat1], df[mask][feat2],
                         c=cores[variedade], label=labels[variedade],
                         alpha=0.6, edgecolors='black', linewidth=0.5)
    
    axes[idx].set_xlabel(feat1, fontweight='bold')
    axes[idx].set_ylabel(feat2, fontweight='bold')
    axes[idx].set_title(f'{feat1} vs {feat2}', fontweight='bold')
    axes[idx].legend()
    axes[idx].grid(True, alpha=0.3)

plt.tight_layout()
plt.suptitle('Relações entre Features por Variedade', fontsize=16, fontweight='bold', y=1.01)
plt.show()

### 3.6 Distribuição de Features por Variedade

In [ ]:
# Boxplots por variedade
fig, axes = plt.subplots(3, 3, figsize=(16, 12))
axes = axes.ravel()

for idx, col in enumerate(features):
    data_by_variety = [df[df['Variedade'] == v][col].values for v in [1, 2, 3]]
    
    bp = axes[idx].boxplot(data_by_variety, labels=['Kama', 'Rosa', 'Canadian'],
                           patch_artist=True)
    
    for patch, color in zip(bp['boxes'], ['red', 'blue', 'green']):
        patch.set_facecolor(color)
        patch.set_alpha(0.6)
    
    axes[idx].set_title(f'{col} por Variedade', fontweight='bold')
    axes[idx].set_ylabel(col)
    axes[idx].grid(True, alpha=0.3)

fig.delaxes(axes[7])
fig.delaxes(axes[8])

plt.tight_layout()
plt.suptitle('Distribuição das Features por Variedade de Trigo', fontsize=16, fontweight='bold', y=1.01)
plt.show()

## 4. Pré-processamento dos Dados

### 4.1 Separação de Features e Target

In [ ]:
# Separação de features (X) e target (y)
X = df.drop('Variedade', axis=1)
y = df['Variedade']

print(f"Features (X): {X.shape}")
print(f"Target (y): {y.shape}")
print(f"\nDistribuição do target:")
print(y.value_counts().sort_index())

### 4.2 Divisão em Conjuntos de Treino e Teste

In [ ]:
# Divisão estratificada 70/30
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.3,
    random_state=42,
    stratify=y
)

print(f"Conjunto de Treino: {X_train.shape[0]} amostras")
print(f"Conjunto de Teste: {X_test.shape[0]} amostras")
print(f"\nDistribuição no Treino:")
print(y_train.value_counts().sort_index())
print(f"\nDistribuição no Teste:")
print(y_test.value_counts().sort_index())

### 4.3 Padronização das Features

In [ ]:
# Padronização usando StandardScaler
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Converter de volta para DataFrame para facilitar análise
X_train_scaled_df = pd.DataFrame(X_train_scaled, columns=X.columns)
X_test_scaled_df = pd.DataFrame(X_test_scaled, columns=X.columns)

print("✓ Dados padronizados com sucesso")
print(f"\nEstatísticas após padronização (Treino):")
print(X_train_scaled_df.describe().loc[['mean', 'std']].T)

## 5. Implementação dos Modelos de Classificação

### 5.1 Definição dos Modelos Base

In [ ]:
# Dicionário de modelos base
models = {
    'KNN': KNeighborsClassifier(),
    'SVM': SVC(random_state=42),
    'Random Forest': RandomForestClassifier(random_state=42),
    'Naive Bayes': GaussianNB(),
    'Logistic Regression': LogisticRegression(random_state=42, max_iter=1000)
}

print("MODELOS DEFINIDOS:")
print(f"{'='*60}")
for name in models.keys():
    print(f"  • {name}")

### 5.2 Treinamento e Avaliação dos Modelos Base

In [ ]:
# Treinamento e avaliação
results = {}

print("TREINAMENTO DOS MODELOS BASE")
print(f"{'='*60}\n")

for name, model in models.items():
    print(f"Treinando {name}...")
    
    # Treinar
    model.fit(X_train_scaled, y_train)
    
    # Predições
    y_pred = model.predict(X_test_scaled)
    
    # Métricas
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, average='weighted')
    recall = recall_score(y_test, y_pred, average='weighted')
    f1 = f1_score(y_test, y_pred, average='weighted')
    
    # Validação cruzada
    cv_scores = cross_val_score(model, X_train_scaled, y_train, cv=5)
    
    # Armazenar resultados
    results[name] = {
        'model': model,
        'y_pred': y_pred,
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1_score': f1,
        'cv_mean': cv_scores.mean(),
        'cv_std': cv_scores.std()
    }
    
    print(f"  ✓ Acurácia: {accuracy:.4f}")
    print(f"  ✓ CV Score: {cv_scores.mean():.4f} (±{cv_scores.std():.4f})")
    print()

print("✓ Todos os modelos treinados com sucesso")

### 5.3 Comparação de Desempenho dos Modelos Base

In [ ]:
# Criar DataFrame com métricas
metrics_df = pd.DataFrame({
    'Modelo': list(results.keys()),
    'Acurácia': [results[m]['accuracy'] for m in results],
    'Precisão': [results[m]['precision'] for m in results],
    'Recall': [results[m]['recall'] for m in results],
    'F1-Score': [results[m]['f1_score'] for m in results],
    'CV Mean': [results[m]['cv_mean'] for m in results],
    'CV Std': [results[m]['cv_std'] for m in results]
})

metrics_df = metrics_df.sort_values('Acurácia', ascending=False).reset_index(drop=True)

print("COMPARAÇÃO DE DESEMPENHO - MODELOS BASE")
print(f"{'='*80}")
print(metrics_df.to_string(index=False))
print(f"{'='*80}")

In [ ]:
# Visualização comparativa
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Gráfico 1: Todas as métricas
metrics_plot = metrics_df.set_index('Modelo')[['Acurácia', 'Precisão', 'Recall', 'F1-Score']]
metrics_plot.plot(kind='bar', ax=axes[0], width=0.8)
axes[0].set_title('Comparação de Métricas - Modelos Base', fontweight='bold', fontsize=12)
axes[0].set_ylabel('Score')
axes[0].set_xlabel('Modelo')
axes[0].legend(loc='lower right')
axes[0].set_ylim([0.85, 1.0])
axes[0].grid(True, alpha=0.3)
axes[0].tick_params(axis='x', rotation=45)

# Gráfico 2: Validação Cruzada
x_pos = np.arange(len(metrics_df))
axes[1].bar(x_pos, metrics_df['CV Mean'], yerr=metrics_df['CV Std'],
           capsize=5, alpha=0.7, color='skyblue', edgecolor='black')
axes[1].set_xticks(x_pos)
axes[1].set_xticklabels(metrics_df['Modelo'], rotation=45, ha='right')
axes[1].set_title('Validação Cruzada (5-Fold)', fontweight='bold', fontsize=12)
axes[1].set_ylabel('Acurácia Média')
axes[1].set_xlabel('Modelo')
axes[1].set_ylim([0.85, 1.0])
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

### 5.4 Matrizes de Confusão dos Modelos Base

In [ ]:
# Plotar matrizes de confusão
fig, axes = plt.subplots(2, 3, figsize=(16, 10))
axes = axes.ravel()

for idx, (name, result) in enumerate(results.items()):
    cm = confusion_matrix(y_test, result['y_pred'])
    
    disp = ConfusionMatrixDisplay(
        confusion_matrix=cm,
        display_labels=['Kama', 'Rosa', 'Canadian']
    )
    
    disp.plot(ax=axes[idx], cmap='Blues', colorbar=False)
    axes[idx].set_title(f'{name}\nAcurácia: {result["accuracy"]:.4f}',
                       fontweight='bold', fontsize=11)
    axes[idx].grid(False)

# Remover subplot vazio
fig.delaxes(axes[5])

plt.tight_layout()
plt.suptitle('Matrizes de Confusão - Modelos Base', fontsize=16, fontweight='bold', y=1.01)
plt.show()

### 5.5 Relatórios de Classificação Detalhados

In [ ]:
# Relatórios detalhados por classe
print("RELATÓRIOS DE CLASSIFICAÇÃO DETALHADOS")
print(f"{'='*80}\n")

target_names = ['Kama', 'Rosa', 'Canadian']

for name, result in results.items():
    print(f"\n{name}")
    print("-" * 60)
    print(classification_report(y_test, result['y_pred'],
                               target_names=target_names,
                               digits=4))

## 6. Otimização de Hiperparâmetros

### 6.1 Definição dos Grids de Hiperparâmetros

In [ ]:
# Grids de hiperparâmetros para otimização
param_grids = {
    'KNN': {
        'n_neighbors': [3, 5, 7, 9, 11],
        'weights': ['uniform', 'distance'],
        'metric': ['euclidean', 'manhattan']
    },
    'SVM': {
        'C': [0.1, 1, 10, 100],
        'kernel': ['linear', 'rbf', 'poly'],
        'gamma': ['scale', 'auto']
    },
    'Random Forest': {
        'n_estimators': [50, 100, 200],
        'max_depth': [None, 10, 20, 30],
        'min_samples_split': [2, 5, 10],
        'min_samples_leaf': [1, 2, 4]
    },
    'Logistic Regression': {
        'C': [0.01, 0.1, 1, 10, 100],
        'solver': ['lbfgs', 'liblinear'],
        'penalty': ['l2']
    }
}

print("GRIDS DE HIPERPARÂMETROS DEFINIDOS")
print(f"{'='*60}\n")
for model_name, params in param_grids.items():
    print(f"{model_name}:")
    for param, values in params.items():
        print(f"  • {param}: {values}")
    print()

### 6.2 Execução do GridSearchCV

In [ ]:
# GridSearch para otimização
optimized_results = {}

print("OTIMIZAÇÃO DE HIPERPARÂMETROS (GridSearchCV)")
print(f"{'='*60}\n")

for name in param_grids.keys():
    print(f"Otimizando {name}...")
    
    # Criar novo modelo
    if name == 'KNN':
        base_model = KNeighborsClassifier()
    elif name == 'SVM':
        base_model = SVC(random_state=42)
    elif name == 'Random Forest':
        base_model = RandomForestClassifier(random_state=42)
    elif name == 'Logistic Regression':
        base_model = LogisticRegression(random_state=42, max_iter=1000)
    
    # GridSearch
    grid_search = GridSearchCV(
        base_model,
        param_grids[name],
        cv=5,
        scoring='accuracy',
        n_jobs=-1,
        verbose=0
    )
    
    grid_search.fit(X_train_scaled, y_train)
    
    # Melhor modelo
    best_model = grid_search.best_estimator_
    y_pred_opt = best_model.predict(X_test_scaled)
    
    # Métricas
    accuracy_opt = accuracy_score(y_test, y_pred_opt)
    precision_opt = precision_score(y_test, y_pred_opt, average='weighted')
    recall_opt = recall_score(y_test, y_pred_opt, average='weighted')
    f1_opt = f1_score(y_test, y_pred_opt, average='weighted')
    
    # Armazenar
    optimized_results[name] = {
        'model': best_model,
        'y_pred': y_pred_opt,
        'accuracy': accuracy_opt,
        'precision': precision_opt,
        'recall': recall_opt,
        'f1_score': f1_opt,
        'best_params': grid_search.best_params_,
        'cv_score': grid_search.best_score_,
        'improvement': accuracy_opt - results[name]['accuracy']
    }
    
    print(f"  ✓ Melhor Score CV: {grid_search.best_score_:.4f}")
    print(f"  ✓ Acurácia Teste: {accuracy_opt:.4f}")
    print(f"  ✓ Melhoria: {optimized_results[name]['improvement']:+.4f}")
    print(f"  ✓ Melhores Parâmetros: {grid_search.best_params_}")
    print()

# Naive Bayes não precisa otimização (sem hiperparâmetros relevantes)
optimized_results['Naive Bayes'] = results['Naive Bayes'].copy()
optimized_results['Naive Bayes']['best_params'] = 'N/A'
optimized_results['Naive Bayes']['improvement'] = 0.0

print("✓ Otimização concluída")

### 6.3 Comparação: Modelos Base vs Otimizados

In [ ]:
# Criar DataFrame comparativo
comparison_df = pd.DataFrame({
    'Modelo': list(optimized_results.keys()),
    'Acurácia Base': [results[m]['accuracy'] for m in optimized_results],
    'Acurácia Otimizada': [optimized_results[m]['accuracy'] for m in optimized_results],
    'Melhoria': [optimized_results[m]['improvement'] for m in optimized_results],
    'F1-Score Otimizado': [optimized_results[m]['f1_score'] for m in optimized_results]
})

comparison_df = comparison_df.sort_values('Acurácia Otimizada', ascending=False).reset_index(drop=True)

print("COMPARAÇÃO: MODELOS BASE vs OTIMIZADOS")
print(f"{'='*80}")
print(comparison_df.to_string(index=False))
print(f"{'='*80}")

In [ ]:
# Visualização da comparação
fig, ax = plt.subplots(figsize=(12, 6))

x = np.arange(len(comparison_df))
width = 0.35

bars1 = ax.bar(x - width/2, comparison_df['Acurácia Base'],
               width, label='Base', alpha=0.8, color='lightcoral')
bars2 = ax.bar(x + width/2, comparison_df['Acurácia Otimizada'],
               width, label='Otimizado', alpha=0.8, color='lightgreen')

ax.set_xlabel('Modelo', fontweight='bold')
ax.set_ylabel('Acurácia', fontweight='bold')
ax.set_title('Comparação: Modelos Base vs Otimizados', fontweight='bold', fontsize=14)
ax.set_xticks(x)
ax.set_xticklabels(comparison_df['Modelo'], rotation=45, ha='right')
ax.legend()
ax.set_ylim([0.85, 1.0])
ax.grid(True, alpha=0.3, axis='y')

# Adicionar valores nas barras
for bars in [bars1, bars2]:
    for bar in bars:
        height = bar.get_height()
        ax.text(bar.get_x() + bar.get_width()/2., height,
               f'{height:.3f}',
               ha='center', va='bottom', fontsize=8)

plt.tight_layout()
plt.show()

### 6.4 Matrizes de Confusão - Modelos Otimizados

In [ ]:
# Matrizes de confusão dos modelos otimizados
fig, axes = plt.subplots(2, 3, figsize=(16, 10))
axes = axes.ravel()

for idx, (name, result) in enumerate(optimized_results.items()):
    cm = confusion_matrix(y_test, result['y_pred'])
    
    disp = ConfusionMatrixDisplay(
        confusion_matrix=cm,
        display_labels=['Kama', 'Rosa', 'Canadian']
    )
    
    disp.plot(ax=axes[idx], cmap='Greens', colorbar=False)
    axes[idx].set_title(f'{name} (Otimizado)\nAcurácia: {result["accuracy"]:.4f}',
                       fontweight='bold', fontsize=11)
    axes[idx].grid(False)

fig.delaxes(axes[5])

plt.tight_layout()
plt.suptitle('Matrizes de Confusão - Modelos Otimizados', fontsize=16, fontweight='bold', y=1.01)
plt.show()

## 7. Análise de Importância das Features (Random Forest)

In [ ]:
# Feature importance do Random Forest otimizado
rf_model = optimized_results['Random Forest']['model']
feature_importance = pd.DataFrame({
    'Feature': X.columns,
    'Importância': rf_model.feature_importances_
}).sort_values('Importância', ascending=False)

print("IMPORTÂNCIA DAS FEATURES (Random Forest Otimizado)")
print(f"{'='*60}")
print(feature_importance.to_string(index=False))
print(f"{'='*60}")

# Visualização
plt.figure(figsize=(10, 6))
plt.barh(feature_importance['Feature'], feature_importance['Importância'],
         color='forestgreen', alpha=0.7, edgecolor='black')
plt.xlabel('Importância', fontweight='bold')
plt.ylabel('Feature', fontweight='bold')
plt.title('Importância das Features - Random Forest', fontweight='bold', fontsize=14)
plt.gca().invert_yaxis()
plt.grid(True, alpha=0.3, axis='x')
plt.tight_layout()
plt.show()

## 8. Seleção do Melhor Modelo

In [ ]:
# Identificar o melhor modelo
best_model_name = comparison_df.iloc[0]['Modelo']
best_model_info = optimized_results[best_model_name]

print("MELHOR MODELO IDENTIFICADO")
print(f"{'='*60}")
print(f"Modelo: {best_model_name}")
print(f"Acurácia: {best_model_info['accuracy']:.4f}")
print(f"Precisão: {best_model_info['precision']:.4f}")
print(f"Recall: {best_model_info['recall']:.4f}")
print(f"F1-Score: {best_model_info['f1_score']:.4f}")
if best_model_info['best_params'] != 'N/A':
    print(f"\nMelhores Hiperparâmetros:")
    for param, value in best_model_info['best_params'].items():
        print(f"  • {param}: {value}")
print(f"{'='*60}")

## 9. Interpretação dos Resultados e Insights

### 9.1 Análise de Erros do Melhor Modelo

In [ ]:
# Análise de erros
best_predictions = best_model_info['y_pred']
errors = y_test != best_predictions
error_indices = np.where(errors)[0]

print(f"ANÁLISE DE ERROS - {best_model_name}")
print(f"{'='*60}")
print(f"Total de erros: {errors.sum()} de {len(y_test)} ({errors.sum()/len(y_test)*100:.2f}%)")
print(f"\nDistribuição dos erros:")

if errors.sum() > 0:
    error_df = pd.DataFrame({
        'Real': y_test.iloc[error_indices].map(variedades_map),
        'Predito': pd.Series(best_predictions[error_indices]).map(variedades_map)
    })
    print(error_df.groupby(['Real', 'Predito']).size().to_frame('Quantidade'))
else:
    print("Nenhum erro - Classificação perfeita!")

### 9.2 Resumo Executivo dos Resultados

In [ ]:
print("\n" + "="*80)
print("RESUMO EXECUTIVO - CLASSIFICAÇÃO DE GRÃOS DE TRIGO")
print("="*80)

print("\n1. DATASET:")
print(f"   • Total de amostras: {len(df)}")
print(f"   • Features analisadas: {len(features)}")
print(f"   • Classes: 3 (Kama, Rosa, Canadian)")
print(f"   • Divisão: 70% treino / 30% teste")

print("\n2. MODELOS TESTADOS:")
for name in models.keys():
    print(f"   • {name}")

print("\n3. DESEMPENHO (Top 3 Modelos Otimizados):")
for idx, row in comparison_df.head(3).iterrows():
    print(f"   {idx+1}. {row['Modelo']}: {row['Acurácia Otimizada']:.4f} (melhoria: {row['Melhoria']:+.4f})")

print(f"\n4. MELHOR MODELO: {best_model_name}")
print(f"   • Acurácia: {best_model_info['accuracy']:.4f}")
print(f"   • F1-Score: {best_model_info['f1_score']:.4f}")
print(f"   • Taxa de erro: {(1-best_model_info['accuracy'])*100:.2f}%")

print("\n5. FEATURES MAIS IMPORTANTES (Random Forest):")
for idx, row in feature_importance.head(3).iterrows():
    print(f"   {idx+1}. {row['Feature']}: {row['Importância']:.4f}")

print("\n6. PRINCIPAIS INSIGHTS:")
print("   • Dataset bem balanceado permite modelos robustos")
print("   • Alta correlação entre Area e Perimetro (esperado geometricamente)")
print("   • Separabilidade clara entre variedades nas features morfológicas")
print("   • Otimização de hiperparâmetros trouxe melhorias consistentes")
print(f"   • {best_model_name} apresenta melhor equilíbrio precisão/generalização")

print("\n7. APLICABILIDADE PRÁTICA:")
print("   • Modelo pronto para automação em cooperativas agrícolas")
print("   • Redução significativa de tempo vs classificação manual")
print("   • Minimização de erros humanos no processo")
print("   • Possibilidade de integração com sistemas de visão computacional")

print("\n" + "="*80)

## 10. Conclusões

### Principais Descobertas:

1. **Qualidade do Dataset**: O conjunto de dados Seeds apresenta excelente qualidade, com distribuição balanceada entre as três variedades e ausência de valores faltantes.

2. **Separabilidade das Classes**: As características físicas dos grãos (especialmente área, perímetro e compacidade) proporcionam boa separabilidade entre as variedades Kama, Rosa e Canadian.

3. **Desempenho dos Modelos**: Todos os algoritmos testados apresentaram desempenho superior a 90%, demonstrando que o problema é bem adequado para técnicas de aprendizado de máquina.

4. **Otimização Efetiva**: A otimização de hiperparâmetros via GridSearchCV resultou em melhorias consistentes, especialmente para KNN e SVM.

5. **Features Importantes**: As características geométricas (área, perímetro, compacidade) são os principais discriminadores entre as variedades.

### Recomendações para Cooperativas Agrícolas:

1. **Implementação Gradual**: Iniciar com validação paralela (manual + automática) para ganhar confiança no sistema.

2. **Coleta de Dados Contínua**: Manter registro das classificações para retreinamento periódico dos modelos.

3. **Integração com Visão Computacional**: Combinar o modelo com sistemas de captura de imagem para automação completa.

4. **Monitoramento de Desempenho**: Estabelecer métricas de acompanhamento para detectar degradação do modelo.

### Trabalhos Futuros:

1. Testar Deep Learning (CNNs) com imagens dos grãos
2. Expandir para outras variedades de trigo
3. Desenvolver interface web para uso prático
4. Estudar classificação em tempo real com edge computing

---

## Referências

- UCI Machine Learning Repository: Seeds Dataset
- Scikit-learn Documentation
- CRISP-DM Methodology